In [ ]:
#Data Ingestion     w1aq    `1q`
from langchain_community.document_loaders import TextLoader
loder=TextLoader("data.txt")
text_documents=loder.load()
text_documents

[Document(metadata={'source': 'data.txt'}, page_content='I have a dream that my four little children will one day live in a nation where they will not be judged by \nthe color of their skin but by the content of their character')]

In [4]:
#PDF Loader
from langchain_community.document_loaders import PyPDFLoader
pdf_loader = PyPDFLoader("data.pdf")
pdf_documents = pdf_loader.load()
pdf_documents

[Document(metadata={'producer': 'Skia/PDF m127', 'creator': 'Chromium', 'creationdate': '2025-08-08T12:07:58+00:00', 'moddate': '2025-08-08T12:07:58+00:00', 'source': 'data.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content='Create a detailed road map to achieve this\nOf course. Here is a detailed roadmap to help you prepare for the AWS Certified Developer -\nAssociate exam. This plan is designed to be interactive. For each topic, I will ask you questions,\nand if you make a mistake, I will provide corrections and explanations.\nThe exam is broken down into four main domains. We will tackle them one by one.\nDomain 1\x00 Development with AWS Services \x0032% of Exam)\nThis is the largest portion of the exam, focusing on how to develop applications using core AWS\nservices.\nI am a software developer and I am preparing for\n"AWS Certified Developer Associate". I want to\nuse perplexity to prepare for the exams. I have the\nfollowing goals\n\x00\x00\x00Prepare for each t

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(pdf_documents)
documents[1:2]

[Document(metadata={'producer': 'Skia/PDF m127', 'creator': 'Chromium', 'creationdate': '2025-08-08T12:07:58+00:00', 'moddate': '2025-08-08T12:07:58+00:00', 'source': 'data.pdf', 'total_pages': 3, 'page': 0, 'page_label': '1'}, page_content="\x00\x00\x00I want perplexity to ask me questions on each topic, and I will answer. If I make any mistake,\ncorrect it and explain.\nRoadmap for AWS Certified Developer - Associate Exam Preparation\nTopic 1\x00 Core AWS Services \x00Compute, Storage, & Database)\nInteractive Session 1\x00 Compute Services: We'll start with Amazon EC2 \x00Elastic\nCompute Cloud), AWS Lambda, and AWS Elastic Beanstalk. I'll ask you about their use\ncases, how to configure them, and best practices for deploying applications.\nInteractive Session 2\x00 Storage Services: We will cover Amazon S3 \x00Simple Storage\nService) and Amazon EBS \x00Elastic Block Store). Our questions will focus on storage\nclasses, performance, and security.\nInteractive Session 3\x00 Database

In [6]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['GOOGLE_API_KEY']=os.getenv("GOOGLE_API_KEY")

In [8]:
#Web Page loader
from langchain_community.document_loaders import WebBaseLoader
import bs4
loader = WebBaseLoader(
    web_path=("https://lilianweng.github.io/posts/2023-06-23-agent/"),
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(
        class_=("post-title", "post-meta")
    ))
)
text_documents = loader.load()
text_documents

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n      LLM Powered Autonomous Agents\n    Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n')]

In [ ]:
#Vector embedding and Vector Store

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import Chroma

# Initialize the embeddings model
embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

db = Chroma.from_documents(pdf_documents[:20], embeddings)

#vector DB
query="List all the domains mentioned in the document"
result = db.similarity_search(query)
result[0].page_content

'Create a detailed road map to achieve this\nOf course. Here is a detailed roadmap to help you prepare for the AWS Certified Developer -\nAssociate exam. This plan is designed to be interactive. For each topic, I will ask you questions,\nand if you make a mistake, I will provide corrections and explanations.\nThe exam is broken down into four main domains. We will tackle them one by one.\nDomain 1\x00 Development with AWS Services \x0032% of Exam)\nThis is the largest portion of the exam, focusing on how to develop applications using core AWS\nservices.\nI am a software developer and I am preparing for\n"AWS Certified Developer Associate". I want to\nuse perplexity to prepare for the exams. I have the\nfollowing goals\n\x00\x00\x00Prepare for each topic that is asked in the exam\n\x00\x00\x00I want perplexity to ask me questions on each topic, and I will answer. If I make any mistake,\ncorrect it and explain.\nRoadmap for AWS Certified Developer - Associate Exam Preparation\nTopic 1\x0

In [ ]:
#FAISS DB

from langchain_community.vectorstores import FAISS
db1 = FAISS.from_documents(pdf_documents[:20], embeddings)

#vector DB
query="List all the domains mentioned in the document"
result = db1.similarity_search(query)
result[0].page_content

"Now that the roadmap is laid out, we can begin.\nAre you ready to start with Domain 1, Topic 1\x00 Core AWS Services? We'll begin with our first\ninteractive session on Compute Services.\nthe root cause.\nTopic 9\x00 Application Optimization\nInteractive Session 15\x00 Performance and Cost Optimization: I will present an\napplication architecture and ask you to identify potential performance bottlenecks and\nopportunities for cost savings."

In [25]:
#Using local ollama and llama2 as the modal
from langchain_community.llms import Ollama
llm_lama = Ollama(model="llama2")
llm_lama

Ollama()

In [19]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# Design ChatPrompt Template

prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the provided context. Think step by step before providing the answer
<context>{context}</context>
Question: {input} """)

In [20]:
##Chain Introduction
##Create stuff document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain = create_stuff_documents_chain(llm, prompt)

#retriver

retriver = db1.as_retriever()
retriver


VectorStoreRetriever(tags=['FAISS', 'GoogleGenerativeAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001D5531EA850>, search_kwargs={})

In [ ]:
#retrival Chain

from langchain.chains import create_retrieval_chain
retrival_chain = create_retrieval_chain(retriver, document_chain)
response = retrival_chain.invoke({"input": "list all the domains from the document"})
response["answer"]

'The provided context is about preparing for the AWS Certified Developer - Associate exam. It doesn\'t contain any information about the movie "Kuch Kuch Hota Hai". Therefore, I cannot answer your question about the actors in that movie based on this context.'